In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
'''from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")'''

'from langchain_google_genai import ChatGoogleGenerativeAI\nllm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")'

In [6]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama-3.1-8b-instant")

## Simple AI Assistant

In [7]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hello. How can I assist you today?
goodbye take care yourself


In [8]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [9]:
store={}

In [10]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [11]:
config = {"configurable": {"session_id": "firstchat"}}

In [12]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [13]:
model_with_memory.invoke(("Hi! I'm aditya"),config=config).content

"Hello Aditya, it's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [14]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Aditya.'

In [15]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [16]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [17]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [18]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided context, here's the information about Llama 3:

Llama 3 is a model version. 

Three important points about Llama 3 are:

1. **Parameter version**: The provided text mentions the "8B parameter version" of Llama 3, which suggests that it has a specific number of parameters, in this case, 8 billion.

2. **Release date**: Llama 3 was released in April 2024, as per the given context.

3. **Usage**: The information states that both services (presumably AI-powered services) use a Llama 3 model, indicating its potential applications and widespread adoption.


## Let's Start with Tools and Agents

In [19]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [20]:
api_wrapper=WikipediaAPIWrapper()

In [21]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [22]:
tool.name

'wikipedia'

In [23]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [24]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [25]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Model Context Protocol
Summary: The Model Context Protocol (MCP) is an open standard, open-source framework introduced by Anthropic in November 2024 to standardize the way artificial intelligence (AI) systems like large language models (LLMs) integrate and share data with external tools, systems, and data sources. MCP provides a universal interface for reading files, executing functions, and handling contextual prompts. Following its announcement, the protocol was adopted by major AI providers, including OpenAI and Google DeepMind.



Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that 

In [30]:
from langchain_community.tools import YouTubeSearchTool

In [31]:
tool2=YouTubeSearchTool()

In [32]:
tool2.name

'youtube_search'

In [33]:
tool2.run("sunny savita")

"['https://www.youtube.com/watch?v=_kpxLkH5vY0&pp=ygUMc3Vubnkgc2F2aXRh0gcJCdgJAYcqIYzv', 'https://www.youtube.com/watch?v=SPNaP4ik9a4&pp=ygUMc3Vubnkgc2F2aXRh']"

In [34]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [35]:
tool3=TavilySearchResults()

C:\Users\91827\AppData\Local\Temp\ipykernel_13932\140006202.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool3=TavilySearchResults()


In [37]:
tool3.invoke({"query": "What happened in the latest burning man floods"})

[{'title': 'Burning Man flooding: What happened to stranded festivalgoers?',
  'url': 'https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331',
  'content': '## ABC News\n\n## Live\n\n## Video\n\n## Shows\n\n## Shop\n\n## Stream on\n\nhulu\n\n# Burning Man flooding: What happened to stranded festivalgoers?\n\nSome 64,000 people were still on site Monday as the exodus began.\n\nTens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend. [...] The festival, held in the Black Rock Desert in northwestern Nevada, was attended by more than 70,000 people last year, and just as many were expected this year. Burning Man began on Aug. 28 and was scheduled to run through Sept. 4.\n\nOne person died at the festival amid the unusual weather conditions, the Pershing County Sheriff\'s Office confirmed Sunday morning in a statement. The death is under investigation. [

In [38]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [ ]:
tool=load_tools(["wikipedia"],llm=llm)

TypeError: unhashable type: 'TavilySearchResults'

In [40]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

C:\Users\91827\AppData\Local\Temp\ipykernel_13932\2645007812.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


In [ ]:
agent.run("What is current GDP of India?")



> Entering new AgentExecutor chain...
Thought: I need to find the current GDP of India, which might involve finding the current year and the GDP data for that year.
Action: wikipedia
Action Input: current GDP of India
Observation: Page: List of Indian states and union territories by GDP
Summary: These are lists of Indian states and union territories by their nominal gross state domestic product (GSDP). GSDP is the sum of all value added by industries within each state or union territory and serves as a counterpart to the national gross domestic product (GDP). As of 2011, the Government accounted for about 21% of the GDP followed by agriculture with 21% and corporate sector at 12%. The remaining 48% is sourced from small proprietorship and partnership companies, unorganized sector and households.

Page: List of countries by GDP (PPP)
Summary: GDP (PPP) means gross domestic product based on purchasing power parity.  
This article includes a list of countries by their forecast estimated